In [25]:
print("hi bacd")

hi bacd


In [26]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
pip install --upgrade openai

   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/226.7 kB ? eta -:--:--
   ----- --------------------------------- 30.7/226.7 kB 325.1 kB/s eta 0:00:01
   -------------------- ----------------- 122.9/226.7 kB 901.1 kB/s eta 0:00:01
   ---------------------------------------- 226.7/226.7 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.6.0
    Uninstalling openai-1.6.0:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\python312\\scripts\\openai.exe'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ : bool = load_dotenv(find_dotenv()) # read local .env file

client : OpenAI = OpenAI()

In [36]:
from openai.types.chat.chat_completion import ChatCompletion

def chat_completion()-> str:
 response : ChatCompletion = client.chat.completions.create(
        messages=[
            {"role": "user", "content": "compose a poem that explains the concept of loop in programming"},
            {"role": "system","content": "you are a poetic assistant, skilled in explaining complex programming concepts with creative flair"}
        ],
        model="gpt-3.5-turbo-1106",
    )
#  print(response.choices[0].message.content)
 return response.choices[0].message.content


In [37]:
print(chat_completion())

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-Yvk3K***************************************dCEB. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
stream = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role":"user", "content":"write a short headline for my cv analyzer AI tool which will read all given pdf cvs and return the coresponding cvs"}],
    stream=True
)
for part in stream:
    print(part.choices[0].delta.content or "")


"
CV
 Analyzer
 AI
:
 Instant
ly
 Extract
 and
 Analy
ze
 PDF
 Res
umes
"



# JSON output

In [ ]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[{"role":"user", "content":"list of 5 famous programming langauges"}, {"role":"system", "content":"you are a helpful developer friendly assistant who always outputs JSON"}],
    response_format={"type":"json_object"}
)
print(response.choices[0].message.content)

{
  "famousProgrammingLanguages": [
    "Java",
    "Python",
    "JavaScript",
    "C++",
    "C#"
  ]
}


# function calling

In [ ]:
import json

def get_country_weather(country: str)->str:
  if "PAKISTAN" in country.upper():
    response = json.dumps({"country":"Pakistan", "temperature":25})
  elif "AMERICA" in country.upper():  
    response = json.dumps({"country":"America", "temperature":18})
  elif "ITALY" in country.upper():  
    response = json.dumps({"country":"Italy", "temperature":7})
  elif "ANTARCTICA" in country.upper():  
    response = json.dumps({"country":"Antarctica", "temperature":1})
  else:
    response = json.dumps({"country":(country), "temperature":"N/A"})

  return response


In [ ]:
pakistan_info = get_country_weather("wuejni")
print(pakistan_info)

{"country": "wuejni", "temperature": "N/A"}


In [ ]:
from openai.types.chat.chat_completion import ChatCompletionMessage, ChatCompletion
# from openai.types.chat.chat_completion import ChatCompletionMessageParam, ChatCompletionMessageParam

def run_conversation(main_request: str)->str:
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": main_request}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_country_weather",
                "description": "Get the current weather in a given country",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "country": {
                            "type": "string",
                            "description": "The name of the country, e.g. Italy, France",
                        },
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message: ChatCompletionMessage = response.choices[0].message
    display("* First Response: ", dict(response_message))

    tool_calls = response_message.tool_calls
    display("* First Reponse Tool Calls: ", list(tool_calls))
     # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_country_weather": get_country_weather,
        }  # only one function in this example, but you can have multiple
        
        messages.append(response_message)  # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("country"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        display("* Second Request Messages: ", list(messages))
        second_response: ChatCompletion = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        print("* Second Response: ", dict(second_response))
        return second_response.choices[0].message.content

In [ ]:
run_conversation("What's the weather like in Italy, America, and Pakistan?")

'* First Response: '

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_tGGJpjy0O5y6lVmLOwH1p5uz', function=Function(arguments='{"country": "Italy"}', name='get_country_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_mm2AOvjf7ffEHPXcXmTb5UPy', function=Function(arguments='{"country": "America"}', name='get_country_weather'), type='function'),
  ChatCompletionMessageToolCall(id='call_nNuILuZ3flm8tEL7n3VgtMal', function=Function(arguments='{"country": "Pakistan"}', name='get_country_weather'), type='function')]}

'* First Reponse Tool Calls: '

[ChatCompletionMessageToolCall(id='call_tGGJpjy0O5y6lVmLOwH1p5uz', function=Function(arguments='{"country": "Italy"}', name='get_country_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_mm2AOvjf7ffEHPXcXmTb5UPy', function=Function(arguments='{"country": "America"}', name='get_country_weather'), type='function'),
 ChatCompletionMessageToolCall(id='call_nNuILuZ3flm8tEL7n3VgtMal', function=Function(arguments='{"country": "Pakistan"}', name='get_country_weather'), type='function')]

TypeError: get_country_weather() got an unexpected keyword argument 'location'